coroutineを理解する

[参考](https://fisproject.jp/2018/12/python-generators-coroutines/)

ポイント：yieldを文ではなく式として使うことができる。

In [52]:
def f1(n):
    for i in range(n):
        x = yield
        print(x)
    print("coroutine finished")

In [53]:
obj = f1(3)
next(obj)   # nextで、最初のyield式まで評価が進む
obj.send(1) # sendで値をyield式に送ることができる。coroutine内は次のyieldまで進む
obj.send(1)
obj.send(1)

1
1
1
coroutine finished


StopIteration: 

generator内のコードが全て実行されるとStopIterationが発生して終了を知らせる。これは通常のgeneratorと同じ

In [54]:
for i in f1(3):
    print(i)

None
None
None
None
None
None
coroutine finished


yieldによって、何らかの値が帰ってきている模様

In [55]:
def f2(n):
    for i in range(n):
        print(f"waiting {i}: ")
        x = yield i
        print(f"from f2: {x}")
obj2 = f2(3)

In [56]:
v = next(obj2)
print(f"from grobal: {v}")

waiting 0: 
from grobal: 0


In [57]:
v = obj2.send(10)
print(f"from grobal: {v}")

from f2: 10
waiting 1: 
from grobal: 1


In [58]:
v = obj2.send(20)
print(f"from grobal: {v}")

from f2: 20
waiting 2: 
from grobal: 2


In [59]:
v = obj2.send(30)
print(f"from grobal: {v}")

from f2: 30


StopIteration: 

上のようにyieldを通じて、generatorの中の処理を途中で止めつつ、generatorの中と外で値を受け渡ししながら処理を進めることができる

### 雑感

In [62]:
x = input()

 100


In [63]:
print(x)

100


### 雑感
例えば、`input`を使うと、プロセスの状態を保ったまま、プロセスの外界からの入力を待つことができる。この状況では、
- 入力を待つ: プロセス全体
- 入力を入れる: ユーザー
という形になる。coroutineを使うと、これと同じような形を、1つのプロセス内で作り出すことができる。

銀行口座をシミュレートしてみる

In [65]:
def bank(init):
    amount = init
    while True:
        val = yield amount
        if val is None:
            raise StopIteration()
        amount += val

b = bank(100)
balance = next(b)
print(f"initial balance={balance}")
for i in range(3):
    balance = b.send(100)
    print(f"current balance={balance}")
b.send(None)

initial balance=100
current balance=200
current balance=300
current balance=400


RuntimeError: generator raised StopIteration